# 🧠 Smart Agent with Long-term Memory

---
## Section 1: Setup

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


### Import ADK components

In [2]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

print("ADK components imported successfully.")

ADK components imported successfully.


### Helper functions

In [3]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("Helper functions defined.")

Helper functions defined.


### Configure Retry Options

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

---
## Section 2: Initialize MemoryService

### Initialize Memory

In [7]:
memory_service = (
    InMemoryMemoryService()
)  

### Add Memory to Agent

In [8]:
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words.",
)

print("Agent created")

Agent created


In [9]:
# Create Session Service
session_service = InMemorySessionService()  # Handles conversations

# Create runner with BOTH services
runner = Runner(
    agent=user_agent,
    app_name="MemoryDemoApp",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)

print("Agent and Runner created with memory support!")

Agent and Runner created with memory support!


---
## Section 3: Ingest Session Data into Memory

In [10]:
await run_session(
    runner,
    "My favorite color is blue-green. Can you write a Haiku about it?",
    "conversation-01",  # Session ID
)


### Session: conversation-01

User > My favorite color is blue-green. Can you write a Haiku about it?
Model: > A teal ocean wave,
Or a frog in summer's pond,
Nature's calm beauty.


In [11]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

# Let's see what's in the session
print("Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text[:60]
        if event.content and event.content.parts
        else "(empty)"
    )
    print(f"  {event.content.role}: {text}...")

Session contains:
  user: My favorite color is blue-green. Can you write a Haiku about...
  model: A teal ocean wave,
Or a frog in summer's pond,
Nature's calm...


In [13]:
await memory_service.add_session_to_memory(session)

print("Session added to memory!")

Session added to memory!


---
## Section 4: Enable Memory Retrieval in Your Agent

### Add Load Memory Tool to Agent

In [14]:
# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
    tools=[
        load_memory
    ],  # Agent now has access to Memory and can search it whenever it decides to!
)

print("Agent with load_memory tool created.")

Agent with load_memory tool created.


### Update the Runner and Test

In [15]:
# Create a new runner with the updated agent
runner = Runner(
    agent=user_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

await run_session(runner, "What is my favorite color?", "color-test")


### Session: color-test

User > What is my favorite color?


Model: > I have noticed that your favorite color is blue-green.


In [17]:
await run_session(runner, "My birthday is on January 19th.", "birthday-session-01")


### Session: birthday-session-01

User > My birthday is on January 19th.


Model: > I see that your birthday is on January 19th.


In [18]:
# Manually save the session to memory
birthday_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="birthday-session-01"
)

await memory_service.add_session_to_memory(birthday_session)

print("Birthday session saved to memory!")

Birthday session saved to memory!


In [20]:
# Test retrieval in a NEW session
await run_session(
    runner, "When is my birthday?", "birthday-session-02"  # Different session ID
)


### Session: birthday-session-02

User > When is my birthday?
Model: > Your birthday is on January 19th.


### Manual Memory Search

In [21]:
# Search for color preferences
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="What is the user's favorite color?"
)

print("Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        text = memory.content.parts[0].text[:80]
        print(f"  [{memory.author}]: {text}...")

Search Results:
  Found 6 relevant memories

  [user]: My favorite color is blue-green. Can you write a Haiku about it?...
  [MemoryDemoAgent]: A teal ocean wave,
Or a frog in summer's pond,
Nature's calm beauty....
  [user]: My birthday is on March 15th....
  [MemoryDemoAgent]: Got it. I'll remember that your birthday is on March 15th....
  [user]: My birthday is on January 19th....
  [MemoryDemoAgent]: I see that your birthday is on January 19th....


---
## Section 5: Automating Memory Storage

In [22]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("Callback created.")

Callback created.


In [23]:
# Agent with automatic memory saving
auto_memory_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="AutoMemoryAgent",
    instruction="Answer user questions.",
    tools=[preload_memory],
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
)

print("Agent created with automatic memory saving!")

Agent created with automatic memory saving!


### Create a Runner and Test The Agent

In [24]:
# Create a runner for the auto-save agent
# This connects our automated agent to the session and memory services
auto_runner = Runner(
    agent=auto_memory_agent,  # Use the agent with callback + preload_memory
    app_name=APP_NAME,
    session_service=session_service,  # Same services from Section 3
    memory_service=memory_service,
)

print("Runner created.")

Runner created.


In [25]:
# Test 1: Tell the agent about a gift (first conversation)
# The callback will automatically save this to memory when the turn completes
await run_session(
    auto_runner,
    "I gifted a new toy to my nephew on his 1st birthday!",
    "auto-save-test",
)

# Test 2: Ask about the gift in a NEW session (second conversation)
# The agent should retrieve the memory using preload_memory and answer correctly
await run_session(
    auto_runner,
    "What did I gift my nephew?",
    "auto-save-test-2",  # Different session ID - proves memory works across sessions!
)


### Session: auto-save-test

User > I gifted a new toy to my nephew on his 1st birthday!
Model: > That's wonderful! A 1st birthday is such a special milestone. I hope your nephew enjoys his new toy!

### Session: auto-save-test-2

User > What did I gift my nephew?
Model: > You gifted your nephew a new toy.


## **Congratulations! 🎉** 